In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Spark') \
    .getOrCreate()

23/05/25 20:37:20 WARN Utils: Your hostname, Nguyen-Duc-Trung resolves to a loopback address: 127.0.1.1; using 192.168.130.52 instead (on interface eth0)
23/05/25 20:37:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/25 20:37:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("header", "true").parquet("../data/yellow_tripdata_2022-01.parquet")

In [4]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-01-01 07:35:40|  2022-01-01 07:53:29|            2.0|          3.8|       1.0|                 N|         142|         236|           1|       14.5|  3.0|    0.5|      3.6

In [5]:
#Check shape of the dataframe
print((df.count(), len(df.columns)))

(2463931, 19)


In [30]:
!wc -l ../data/yellow_tripdata_2022-01.parquet

122909 ../data/yellow_tripdata_2022-01.parquet


In [6]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [32]:
type(df)

pyspark.sql.dataframe.DataFrame

In [33]:
df.head(5)

[Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2022, 1, 1, 7, 35, 40), tpep_dropoff_datetime=datetime.datetime(2022, 1, 1, 7, 53, 29), passenger_count=2.0, trip_distance=3.8, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=142, DOLocationID=236, payment_type=1, fare_amount=14.5, extra=3.0, mta_tax=0.5, tip_amount=3.65, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=21.95, congestion_surcharge=2.5, airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2022, 1, 1, 7, 33, 43), tpep_dropoff_datetime=datetime.datetime(2022, 1, 1, 7, 42, 7), passenger_count=1.0, trip_distance=2.1, RatecodeID=1.0, store_and_fwd_flag='N', PULocationID=236, DOLocationID=42, payment_type=1, fare_amount=8.0, extra=0.5, mta_tax=0.5, tip_amount=4.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=13.3, congestion_surcharge=0.0, airport_fee=0.0),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2022, 1, 1, 7, 53, 21), tpep_dropoff_datetime=datetime.dateti

In [7]:
#Repartition the dataframe to 10 partitions
df = df.repartition(10)


In [8]:
#Check the number of partitions
df.rdd.getNumPartitions()

10

In [37]:
#Check the size of each partition
df.rdd.glom().map(len).collect()



[246393,
 246393,
 246393,
 246393,
 246394,
 246393,
 246393,
 246393,
 246393,
 246393]

In [9]:
#Save the partitioned dataframe as parquet file
df.write.parquet("../partitions/yellow/2022/01/")

23/05/24 14:42:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/05/24 14:42:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/05/24 14:42:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


23/05/24 14:42:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/05/24 14:42:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [4]:
sp_df = spark.read.parquet("../partitions/yellow/2022/01/")

In [5]:
type(sp_df)

pyspark.sql.dataframe.DataFrame

In [6]:
sp_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [7]:
#Create the duration column by subtracting the dropoff time from the pickup time
from pyspark.sql.functions import col, unix_timestamp, round

sp_df = sp_df.withColumn("duration", round((unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime")))/60))

In [10]:
#Sort the dataframe by duration in ascending order
sp_df.select("duration").sort(col("duration").desc()).show()

+--------+
|duration|
+--------+
|  8513.0|
|  7891.0|
|  7872.0|
|  4284.0|
|  4019.0|
|  3923.0|
|  2871.0|
|  2686.0|
|  2680.0|
|  1974.0|
|  1849.0|
|  1723.0|
|  1723.0|
|  1463.0|
|  1440.0|
|  1440.0|
|  1440.0|
|  1440.0|
|  1440.0|
|  1440.0|
+--------+
only showing top 20 rows



In [11]:
#Count the number of row in duration column
sp_df.select("duration").count()

2463931

In [12]:
#Count the number of null values in duration column
sp_df.select("duration").filter(col("duration").isNull()).count()

0

In [13]:
#Display the distribution of duration column
sp_df.select("duration").describe().show()

+-------+------------------+
|summary|          duration|
+-------+------------------+
|  count|           2463931|
|   mean|14.219502088329584|
| stddev|  46.4466802662856|
|    min|           -3442.0|
|    max|            8513.0|
+-------+------------------+



In [14]:
#Remove the outliers from the duration column which are not belong to the range of 1 to 60 minutes
sp_df = sp_df.filter((col("duration") >= 1) & (col("duration") <= 60))


In [15]:
#Display the distribution of duration column again
sp_df.select("duration").describe().show()

+-------+------------------+
|summary|          duration|
+-------+------------------+
|  count|           2429609|
|   mean|12.670490189985301|
| stddev|  9.07449437070349|
|    min|               1.0|
|    max|              60.0|
+-------+------------------+



In [17]:
2429609/2463931*100

98.60702268042408

In [16]:
#Check the percentage duration data remaining after removing the outliers
(sp_df.select("duration").count()/df.select("duration").count())*100

NameError: name 'df' is not defined

In [20]:
#Check the number of cores available in the system
!nproc --all

16
